## Setup Environment <br>
<br>
• Torch - 1.6.0<br>
• Torchvision – 0.7.0<br>
• CUDA – 10.1<br>
• CuDNN – v7.6.5.32 for CUDA 10.1<br>
• Sklearn – 0.22.2.post1<br>
• MLFlow – 1.10.0<br>
• numpy – 1.18.5<br>

In [2]:
import torch
import torch.nn as nn
from torch.utils import data
import torchvision
import torchvision.datasets
import sklearn
from sklearn.metrics import roc_auc_score, accuracy_score
import numpy as np 
import mlflow
import mlflow.pytorch

Print versions

In [3]:
print("PyTorch: {}".format(torch.__version__))
print("torchvision: {}".format(torchvision.__version__))
print("sklearn: {}".format(sklearn.__version__))
print("MLFlow: {}".format(mlflow.__version__))
print("Numpy: {}".format(np.__version__))
print("Device: ", device)

PyTorch: 2.0.0
torchvision: 0.15.1
sklearn: 1.2.2
MLFlow: 2.2.2
Numpy: 1.23.5
Device:  cpu


In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available()
else "cpu")

### Data Preparation

Define basic Hyperparameters

In [11]:
batch_size = 256
num_classes = 10
learning_rate = 0.001

Train and test sets

In [5]:
train_set = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=None)
test_set = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=None)


13.9%

100.0%


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100.0%


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




73.5%

100.0%


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100.0%

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



Seperate data and labels

In [6]:
x_train, y_train = train_set.data, train_set.targets
x_test, y_test = test_set.data, test_set.targets

the format of the data should be (m, c, h, w), where m stands for the number of samples, c stands for the number of channels, h stands for the height of the samples, and w stands for the width of the samples.

In [7]:
x_train = x_train.reshape(x_train.shape[0],1, x_train.shape[1], x_train.shape[2])
x_test = x_test.reshape(x_test.shape[0],1, x_test.shape[1], x_test.shape[2])

In [8]:
y_train[0]

tensor(5)

In [9]:

def to_one_hot(num_classes, labels):
    one_hot = torch.zeros(([labels.shape[0], num_classes])) 
    for f in range(len(labels)):
        one_hot[f][labels[f]] = 1 
    return one_hot

In [12]:
y_train = to_one_hot(num_classes, y_train)
y_test = to_one_hot(num_classes, y_test)

In [13]:
print("Shapes")
print("x_train: {}\ny_train: {}".format(x_train.shape,
y_train.shape))
print("x_test: {}\ny_test: {}".format(x_test.shape,
y_test.shape))

Shapes
x_train: torch.Size([60000, 1, 28, 28])
y_train: torch.Size([60000, 10])
x_test: torch.Size([10000, 1, 28, 28])
y_test: torch.Size([10000, 10])


## MLFlow Run- Training and Evaluating

In [14]:
class model(nn.Module): 
    def __init__(self):
        super(model, self).__init__()
        # IN 1x28x28 OUT 16x14x14
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=2, padding=1, dilation=1)
        # IN 16x14x14 OUT 32x6x6
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=0, dilation=1)
        # IN 32x6x6 OUT 64x2x2
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=2, padding=0, dilation=1)
        # IN 64x2x2 OUT 256
        self.flat1 = nn.Flatten()
        self.dense1 = nn.Linear(in_features=256, out_features=128)
        self.dense2 = nn.Linear(in_features=128, out_features=64)
        self.dense3 = nn.Linear(in_features=64, out_features=10)


    def forward(self, x): 
        x = self.conv1(x) 
        x = nn.ReLU()(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.conv3(x)
        x = nn.ReLU()(x)
        x = self.flat1(x)
        x = self.dense1(x)
        x = nn.ReLU()(x)
        x = self.dense2(x)
        x = nn.ReLU()(x)
        x = self.dense3(x)
        x = nn.Softmax()(x)
        return x

In [15]:
model = model().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCELoss()

In [16]:
dataset = data.TensorDataset(x_train,y_train)
train_loader = data.DataLoader(dataset, batch_size=batch_size)

In [21]:
num_epochs = 5
for f in range(num_epochs):
    for batch_num, minibatch in enumerate(train_loader):
        minibatch_x, minibatch_y = minibatch[0], minibatch[1]
        output = model.forward(torch.Tensor(minibatch_x.float()))
        loss = criterion(output, torch.Tensor(minibatch_y.float()))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f"Epoch {f} Batch_Num {batch_num} Loss {loss}")


/var/folders/b0/lvr6c0md2rb5v_rb1ngcc27h0000gn/T/ipykernel_10277/2201976014.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = nn.Softmax()(x)


Epoch 0 Batch_Num 0 Loss 0.008782649412751198
Epoch 0 Batch_Num 1 Loss 0.007078584283590317
Epoch 0 Batch_Num 2 Loss 0.0029490473680198193
Epoch 0 Batch_Num 3 Loss 0.008408360183238983
Epoch 0 Batch_Num 4 Loss 0.0056398650631308556
Epoch 0 Batch_Num 5 Loss 0.009857377037405968
Epoch 0 Batch_Num 6 Loss 0.009978807531297207
Epoch 0 Batch_Num 7 Loss 0.0054803951643407345
Epoch 0 Batch_Num 8 Loss 0.003967866767197847
Epoch 0 Batch_Num 9 Loss 0.00782245397567749
Epoch 0 Batch_Num 10 Loss 0.007897497154772282
Epoch 0 Batch_Num 11 Loss 0.008148512803018093
Epoch 0 Batch_Num 12 Loss 0.0014904446434229612
Epoch 0 Batch_Num 13 Loss 0.009080146439373493
Epoch 0 Batch_Num 14 Loss 0.012505291029810905
Epoch 0 Batch_Num 15 Loss 0.005822139326483011
Epoch 0 Batch_Num 16 Loss 0.004930921830236912
Epoch 0 Batch_Num 17 Loss 0.005538105498999357
Epoch 0 Batch_Num 18 Loss 0.00564067717641592
Epoch 0 Batch_Num 19 Loss 0.009290741756558418
Epoch 0 Batch_Num 20 Loss 0.00917443074285984
Epoch 0 Batch_Num 21 L

In [22]:
%%time
mlflow.set_experiment("PyTorch_MNIST") 
with mlflow.start_run():
    preds = model.forward(torch.Tensor(x_test.float()))
    preds = np.round(preds.detach().cpu().numpy())
    eval_acc = accuracy_score(y_test, preds)
    auc_score = roc_auc_score(y_test, preds)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("num_epochs", num_epochs)
    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_metric("eval_acc", eval_acc)
    mlflow.log_metric("auc_score", auc_score)
    print("eval_acc: ", eval_acc)
    print("auc_score: ", auc_score)
    mlflow.pytorch.log_model(model, "PyTorch_MNIST")
mlflow.end_run()

2023/03/20 23:31:09 INFO mlflow.tracking.fluent: Experiment with name 'PyTorch_MNIST' does not exist. Creating a new experiment.
/var/folders/b0/lvr6c0md2rb5v_rb1ngcc27h0000gn/T/ipykernel_10277/2201976014.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = nn.Softmax()(x)


eval_acc:  0.9789
auc_score:  0.9883053250821299
CPU times: user 1.91 s, sys: 681 ms, total: 2.59 s
Wall time: 4.27 s


/Users/alimokh/opt/miniconda3/envs/torch/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
